# QICK SOC216 Hardware Testbook
This notebook contains a test plan for the SOC216 board.

To work with this notebook you need to run the support function cell and the firmware initialization cell.

## 1. Support functions for notebook
Please run the following cell to import a number of helper functions to be used within the environment.

In [ ]:
### Support functions required by test notebook ###
print("Please wait while support functions are imported...\n")

import contextlib
import time

import numpy as np
from numpy.fft import fft
import matplotlib.pyplot as plt
from pynq import allocate

from qick.rfboard import AdcDcCard216, AdcRfCard216, DacDcCard216, DacRfCard216


def get_card_slot_and_type(soc, dac_channel, adc_channel):
    """Make sure the selected channels are for the right type of card
    If no specific class is required, Object will ensure the card is not None
    
    Args:
        soc: instance of QICK SOC
        dac_channel: int of DAC channel
        adc_channel: int or None for ADC channel
    """
    dac_card_slot = dac_channel // 4
    dac_card_type = type(soc.dac_cards[dac_card_slot])

    if adc_channel is not None:
        adc_card_slot = adc_channel // 2
        adc_card_type = type(soc.adc_cards[adc_card_slot])

    else:
        adc_card_slot = None
        adc_card_type = type(None)

    return {
        "adc_channel": adc_channel,
        "dac_channel": dac_channel,
        "adc_card_slot": adc_card_slot,
        "adc_card_type": adc_card_type,
        "dac_card_slot": dac_card_slot,
        "dac_card_type": dac_card_type,
    }


def reset_environment(soc, dac_channel, adc_channel):
    """Ensure the DAC and ADC cards are set to known state (defaults)
        
    Args:
        soc: instance of QICK SOC
        dac_channel: int of DAC channel
        adc_channel: int or None for ADC channel
    """
    print(f"Setting cards DAC Channel:{dac_channel} and ADC Channel:{adc_channel} to known (default) state.")

    card_info = get_card_slot_and_type(soc, dac_channel, adc_channel)

    dac_card_slot = card_info["dac_card_slot"]
    adc_card_slot = card_info["adc_card_slot"]

    ch_map = {
        "dac": dict(enumerate(soc["dacs"])),
        "adc": dict(enumerate(soc["adcs"])),
    }

    dac_chain = soc.dac_cards[dac_card_slot].chains[dac_channel % 4]
    dac_fs = soc["dacs"][ch_map["dac"][dac_channel]]["fs"]

    if adc_channel is not None:
        adc_chain = soc.adc_cards[adc_card_slot].chains[adc_channel % 2]
        adc_fs = soc["adcs"][ch_map["adc"][adc_channel]]["fs"]
    else:
        adc_chain = None
        adc_fs = None

    with contextlib.suppress(AttributeError):
        dac_chain.enable_rf(att1=30, att2=30)
        print(f"-->Setting attenuators to `30` for DAC Channel:{dac_channel}.")
        if adc_chain is not None:
            adc_chain.enable_rf(att=30)
            print(f"-->Setting attenuator to `30` for ADC Channel:{adc_channel}.")

    if adc_chain is not None:
        with contextlib.suppress(AttributeError):
            adc_chain.enable_dc(gain=0)
            print(f"-->Setting DC gain to `0` for ADC Channel:{adc_channel}.")

    with contextlib.suppress(AttributeError):
        soc.rf.set_nyquist(ch_map["dac"][dac_channel], 1, blocktype="dac")
        print(f"-->Setting nyquist zone to `1` on DAC Channel:{dac_channel}.")
        if adc_channel is not None:
            print(f"-->Setting nyquist zone to `1` on ADC Channel:{adc_channel}.")
            soc.rf.set_nyquist(ch_map["adc"][adc_channel], 1, blocktype="adc")

    configure_signal_generator(soc, dac_channel, dac_fs, output_freq=0, signal_gain=0, disable=True)

    return {
        "ch_map": ch_map,
        "dac_chain": dac_chain,
        "adc_chain": adc_chain,
        "dac_fs": dac_fs,
        "adc_fs": adc_fs,
    }


def configure_signal_generator(soc, dac_channel, dac_fs, output_freq=0, signal_gain=1, disable=False):
    """Setup the signal generator with our expected output
        
    Args:
        soc: instance of QICK SOC
        dac_channel: int of DAC channel
        dac_fs: int showing the DAC frequency spectrum
        output_freq: int of frequency to generate
        signal_gain: float for how much gain to provide
        disable: boolean to switch the generator on/off
    """
    if not disable:
        print(f"Configuring signal generator for DAC Channel:{dac_channel}.")

    try:
        soc.axis_signal_gen_v6_c_0.configure(dac_fs, soc.axis_signal_gen_v6_0)
        soc.axis_signal_gen_v6_c_0.add(freq=output_freq, gain=signal_gain)
        soc.axis_switch_v1_0.sel(mst=dac_channel)

        if disable:
            print("-->Signal generator is powered down.")
            soc.mr_buffer_et_0.dr_start_reg = 0
            soc.mr_buffer_et_0.disable()
        else:
            print(f"-->Signal generator is sending frequency:{output_freq} at gain:{signal_gain} to DAC.")
    except Exception:
        # set everything to "off"
        soc.axis_signal_gen_v6_c_0.add(freq=0)
        soc.mr_buffer_et_0.dr_start_reg = 0
        soc.mr_buffer_et_0.disable()
        raise


def read_data(soc, dac_channel, adc_channel, read_buffer):
    """Read in the data from the ADC channel and disable signal

        Args:
            soc: instance of QICK SOC
            dac_channel: int of DAC channel
            adc_channel: int of ADC channel
            read_buffer: pynq.buffer.PynqBuffer to store the data
    """
    try:
        # setup readout switch
        soc.axis_switch_1.sel(slv=adc_channel)
        # let buffers catch up
        time.sleep(1)
        soc.mr_buffer_et_0.enable()
        time.sleep(0.1)
        soc.mr_buffer_et_0.disable()

        # Read in signal
        print(f"Reading signal from ADC Channel:{adc_channel}.")
        soc.mr_buffer_et_0.enable()
        time.sleep(0.1)  # Simulate acquisition time
        soc.mr_buffer_et_0.disable()

        # Copy signal from buffer/dma to read_buffer
        soc.mr_buffer_et_0.dr_start_reg = 1
        soc.axi_dma_0.recvchannel.transfer(read_buffer)
        soc.axi_dma_0.recvchannel.wait()
        soc.mr_buffer_et_0.dr_start_reg = 0
    finally:
        # make sure to clean up everything
        reset_environment(soc, dac_channel, adc_channel)


def analyze_signal(signal, adc_fs):
    """Gather important features of the signal
    
    Args:
        signal: list of data points
        adc_fs: int showing the ADC frequency spectrum
    """
    if not isinstance(signal, (list, np.ndarray)):
        raise TypeError("Signal must be a list or numpy array.")
    if len(signal) == 0:
        raise ValueError("Signal cannot be empty.")

    print("Running signal analysis...")

    # Apply windowing function
    window = np.hanning(len(signal))
    windowed_signal = signal * window

    # Compute FFT and magnitude in dB
    fft_result = fft(windowed_signal) / len(signal)
    fft_magnitude_dB = 20 * np.log10(np.abs(fft_result))
    frequency_axis = np.linspace(0, adc_fs, len(fft_result))

    # Find peak frequency and noise analysis
    max_index = np.argmax(fft_magnitude_dB)
    max_frequency_hz = round(frequency_axis[max_index], 2)
    max_frequency_dB = round(fft_magnitude_dB[max_index], 2)

    # Compute noise range
    median_noise_dB = np.median(fft_magnitude_dB)
    background_noise_values = fft_magnitude_dB[fft_magnitude_dB < median_noise_dB]
    mean_noise_dB = round(np.mean(background_noise_values), 2)
    mean_positive_noise_dB = round(np.mean(fft_magnitude_dB[fft_magnitude_dB > 0]), 2)
    mean_negative_noise_dB = round(np.mean(fft_magnitude_dB[fft_magnitude_dB < 0]), 2)

    # Determine Db range for plot
    min_dB = np.floor(np.min(fft_magnitude_dB) / 10) * 10
    max_dB = np.ceil(np.max(fft_magnitude_dB) / 10) * 10

    return {
        "fft_magnitude_dB": fft_magnitude_dB,
        "frequency_axis": frequency_axis,
        "max_index": max_index,
        "max_frequency_hz": max_frequency_hz,
        "max_frequency_dB": max_frequency_dB,
        "mean_noise_dB": mean_noise_dB,
        "mean_positive_noise_dB": mean_positive_noise_dB,
        "mean_negative_noise_dB": mean_negative_noise_dB,
        "min_dB": min_dB,
        "max_dB": max_dB,
        "adc_fs": adc_fs,
        "signal": signal,
    }

def create_plot(datapoints, dac_channel, adc_channel, card_type):
    """Create a standardized plot for card test results
    
    Args:
        datapoints: Dictionary containing signal analysis data
        dac_channel: DAC channel used for testing
        adc_channel: ADC channel used for testing
        card_type: Either "rf" or "dc"
    """
    print("Generating plot...")
    plt.figure(figsize=(12, 6))
    
    # Plot the main frequency data
    plt.plot(datapoints["frequency_axis"], datapoints["fft_magnitude_dB"], color="blue")
    
    # Set up common plot elements
    plt.xlim([0, datapoints["adc_fs"] / 2])
    plt.title(f"{card_type.upper()} Frequency Test of OUTPUT:{dac_channel} into INPUT:{adc_channel}")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude (dB)")
    plt.yticks(np.arange(datapoints["min_dB"] + 10, datapoints["max_dB"] + 10, 10))
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    
    # Add card-specific reference lines
    if card_type.lower() == "rf":
        # Add noise reference lines for RF cards
        reference_lines = [
            (datapoints["mean_noise_dB"], "orange", "Noise (Mean)"),
            (datapoints["mean_positive_noise_dB"], "red", "Mean Positive Noise"),
            (datapoints["mean_negative_noise_dB"], "red", "Mean Negative Noise")
        ]
        
        for value, color, label in reference_lines:
            plt.axhline(value, color=color, linestyle="--", label=label)
    elif card_type.lower() == "dc":
        # Add max frequency line for DC cards
        plt.axhline(datapoints["max_frequency_dB"], color="green", label="Max Frequency")
    else:
        raise ValueError("Wrong card type specified")
    
    # Mark the peak frequency
    plt.scatter(
        datapoints["frequency_axis"][datapoints["max_index"]],
        datapoints["fft_magnitude_dB"][datapoints["max_index"]],
        color="black",
        s=50,
        marker="o",
        label="Spike Frequency"
    )
    
    plt.legend()
    return plt

def card_test(soc, dac_channel, adc_channel, output_freq, signal_gain, output_nqz, input_nqz):
    """Run the selected channels through the test workflow
    
    Args:
        soc: instance of QICK SOC
        dac_channel: int of DAC channel
        adc_channel: int or None for ADC channel
        output_freq: int of frequency to generate
        signal_gain: float for how much gain to provide
        output_nqz: int output nyquist zone
        input_nqz: int input nyquist zone
    """
    card_info = get_card_slot_and_type(soc, dac_channel, adc_channel)
    board_info = reset_environment(soc, dac_channel, adc_channel)

    if adc_channel is not None:
        adc_card_slot = card_info["adc_card_slot"]
        adc_chain = soc.adc_cards[adc_card_slot].chains[adc_channel % 2]
    else:
        adc_chain = None

    if output_freq > board_info["dac_fs"]:
        raise ValueError(f"Selected frequency {output_freq} exceeds DAC limit {board_info['dac_fs']}.")

    print("Setting up test conditions.")
    print(f"-->DAC Channel:{dac_channel} nyquist_zone:{output_nqz}")
    soc.rf.set_nyquist(board_info["ch_map"]["dac"][dac_channel], output_nqz, blocktype="dac")
    if adc_channel is not None:
        print(f"-->ADC Channel:{adc_channel} nyquist_zone:{input_nqz}")
        soc.rf.set_nyquist(board_info["ch_map"]["adc"][adc_channel], input_nqz, blocktype="adc")

    with contextlib.suppress(AttributeError):
        board_info["dac_chain"].set_filter(fc=output_freq / 1_000, bw=1, ftype="bandpass")
        print(f"-->DAC Chain filter fc={output_freq / 1_000} bw=1, ftype=bandpass")
        if adc_channel is not None:
            board_info["adc_chain"].set_filter(fc=output_freq / 1_000, bw=1, ftype="bandpass")
            print(f"-->ADC Chain filter fc={output_freq / 1_000} bw=1, ftype=bandpass")

    with contextlib.suppress(AttributeError):
        board_info["dac_chain"].enable_rf(att1=25, att2=23)
        print(f"-->DAC Channel:{dac_channel} att1=25 att2=23")
        if adc_channel is not None:
            board_info["adc_chain"].enable_rf(att=21)
            print(f"-->ADC Channel:{adc_channel} att=21")

    if adc_channel is not None:
        with contextlib.suppress(RuntimeError, AttributeError):
            adc_chain.enable_dc(gain=7)
            print(f"-->ADC Channel:{adc_channel} DC gain=7")

    configure_signal_generator(soc, dac_channel, board_info["dac_fs"], output_freq, signal_gain)

    if adc_channel is None:
        print("")
        print("Signal generated, no ADC set to read in value.")
        print("Press ENTER to terminate signal:")
        input()
        print("")
        configure_signal_generator(soc, dac_channel, board_info["dac_fs"], disable=True)
        reset_environment(soc, dac_channel, adc_channel)
        return {"data_buffer": [0], "adc_fs": 0}

    read_buffer = allocate(shape=soc.mr_buffer_et_0["maxlen"], dtype=np.int16)
    read_data(soc, dac_channel, adc_channel, read_buffer)

    return {"data_buffer": read_buffer[0:], "adc_fs": board_info["adc_fs"]}


def test_rf_card(soc, dac_channel, adc_channel=None):
    """Run the test workflow for a RF card
    The output frequency of 4_000 was selected "at random"

    Args:
        soc: instance of QICK SOC
        dac_channel: int of DAC channel
        adc_channel: int or None for ADC channel
    """
    print("Checking cards are the correct type.")
    card_info = get_card_slot_and_type(soc, dac_channel, adc_channel)

    if not isinstance(soc.dac_cards[card_info["dac_card_slot"]], DacRfCard216):
        raise ValueError(f"DAC channel {dac_channel} targets slot {card_info['dac_card_slot']} which is not a valid {type(DacRfCard216)} card - found {card_info['dac_card_type']}.")

    if adc_channel is not None and not isinstance(soc.adc_cards[card_info["adc_card_slot"]], AdcRfCard216):
        raise ValueError(f"ADC channel {adc_channel} targets slot {card_info['adc_card_slot']} which is not a valid {type(AdcRfCard216)} card - found {card_info['adc_card_type']}.")

    raw_data = card_test(soc, dac_channel, adc_channel, 4_000, 0.99, 1, 2)

    if adc_channel is not None:
        datapoints = analyze_signal(raw_data["data_buffer"], raw_data["adc_fs"])
        plt = create_plot(datapoints, dac_channel, adc_channel, card_type='rf')

        plt.show()
        # Print frequency and noise details
        print(f"OUTPUT:{dac_channel} is {card_info['dac_card_type']}.")
        print(f"INPUT:{adc_channel} is {card_info['adc_card_type']}.")
        print(f"Raw Signal: {datapoints['signal']}") 
        print(f"Max: {datapoints['max_frequency_hz']}Hz @ {datapoints['max_frequency_dB']}dB.")
        print(f"Background Noise (Hz): {datapoints['mean_negative_noise_dB']} to {datapoints['mean_positive_noise_dB']} => Average {datapoints['mean_noise_dB']}.")

        # Validate frequency and dB range
        # hard coded range based on test frequency and attenuators
        if not (905 <= datapoints["max_frequency_hz"] <= 925):
            raise ValueError(f"{datapoints['max_frequency_hz']}Hz is not within the expected range (905 <= max_frequency_hz <= 925).")

        if not (60 <= datapoints["max_frequency_dB"] <= 70):
            raise ValueError(f"{datapoints['max_frequency_dB']}dB is not within the expected range (60 <= max_frequency_dB <= 70).")


def test_dc_card(soc, dac_channel, adc_channel=None):
    """Run the test workflow for a DC card
    The output frequency of 100 was selected "at random"

    Args:
        soc: instance of QICK SOC
        dac_channel: int of DAC channel
        adc_channel: int or None for ADC channel
    """
    print("Checking cards are the correct type.")
    card_info = get_card_slot_and_type(soc, dac_channel, adc_channel)

    if not isinstance(soc.dac_cards[card_info["dac_card_slot"]], DacDcCard216):
        raise ValueError(f"DAC channel {dac_channel} targets slot {card_info['dac_card_slot']} which is not a valid {type(DacDcCard216)} card - found {card_info['dac_card_type']}.")

    if adc_channel is not None and not isinstance(soc.adc_cards[card_info["adc_card_slot"]], AdcDcCard216):
        raise ValueError(f"ADC channel {adc_channel} targets slot {card_info['adc_card_slot']} which is not a valid {type(AdcDcCard216)} card - found {card_info['adc_card_type']}.")

    raw_data = card_test(soc, dac_channel, adc_channel, 100, 0.5, 1, 1)

    if adc_channel is not None:
        datapoints = analyze_signal(raw_data["data_buffer"], raw_data["adc_fs"])
        plt = create_plot(datapoints, dac_channel, adc_channel, card_type='dc')

        plt.show()
        # Print frequency and noise details
        print(f"OUTPUT:{dac_channel} is {card_info['dac_card_type']}.")
        print(f"INPUT:{adc_channel} is {card_info['adc_card_type']}.")
        print(f"Raw Signal: {datapoints['signal']}") 
        print(f"Max: {datapoints['max_frequency_hz']}Hz @ {datapoints['max_frequency_dB']}dB.")

        # Validate frequency and dB range
        # hard coded range based on test frequency and attenuators
        if not (99 <= datapoints["max_frequency_hz"] <= 101):
            raise ValueError(f"{datapoints['max_frequency_hz']}Hz is not within the expected range (99 <= max_frequency_hz <= 101).")

        if not (70 <= datapoints["max_frequency_dB"] <= 75):
            raise ValueError(f"{datapoints['max_frequency_dB']}dB is not within the expected range (70 <= max_frequency_dB <= 75).")

#
print("Support functions loaded.")

## 2. Firmware initialization
Please run the following cell to setup the SOC with the required firmware.

In [ ]:
print("Please wait for firmware to initialize...\n")

from qick.rfboard import RFQickSoc216V1

FIRMWARE = './test_rf216_v2p2_kt.bit'

soc = RFQickSoc216V1(FIRMWARE, clk_output=None, no_tproc=True)

print(f"Firmware {FIRMWARE} loaded:")
print(soc)

## 3. Test Board Functions
The following cells implement various tests of the SOC and its features.

In [ ]:
### Blink the LEDs in a pattern ###

import time

from tqdm.auto import tqdm

def run_led_pattern(soc, pattern, delay=0.2, offset=0):
    """Set the system LEDs to a pattern.
    This can be used to display any sequence of valid codes.
    Parameters
    ----------
    soc: QICK RF board
        An instance of a class with the attribute pmod_led_gpio.write
    pattern: list, elements of list are either int or bytes
        The sequence of values to write out to the LEDs
    delay : float
        Time (in seconds) to wait between values
    offset : int
        Address to write to for soc.pmod_led_gpio.write
    """
    for value in tqdm(pattern, desc="specific pattern"):
        soc.pmod_led_gpio.write(offset=offset, value=value)
        time.sleep(delay)

bounce_right = [0x80, 0x40, 0x20, 0x10, 0x08, 0x04, 0x02, 0x01]
bounce_left = bounce_right[::-1]
bounces = (bounce_right + bounce_left)*2

try:
    for patterns in tqdm([bounces, [0x00, 0xFF, 0x55, 0xAA, 0xFF, 0x00]], desc="All patterns"):
        run_led_pattern(soc, pattern=patterns)
finally:
    run_led_pattern(soc, pattern=[0x00])

In [ ]:
### Create a ramp up on each DAC Bias (built in port) ###

import time

from tqdm.auto import tqdm
import numpy as np

print("Reminder: the values shown on your scope are how you validate the behavior")

try:
    for count in tqdm(range(8)):
        for voltage in np.linspace(-10, 10, 2000):
            for bias in soc.biases:
                bias.set_volt(voltage)
finally:
    for bias in soc.biases:
        bias.set_volt(0)

In [ ]:
### Read and write with the I/O pmod port (Ribbon Cable) ### 

from tqdm.auto import tqdm

print("Reminder: you should be using a loop back cable to validate the behavior")

offsets = [0x0]
try:
    for offset in offsets:
        for value in tqdm([0x00, 0x01, 0x02, 0x03]): # pmod I/O port is 4 bits wide statically configured
            soc.pmod_bits_gpio.write(offset=offset, value=value)
            found = soc.pmod_bits_gpio.read(offset=offset)
            if found != value:
                raise ValueError(f"{hex(offset)} had value {hex(found)} rather than {hex(value)}")
    print("No errors found")
finally:
    for offset in offsets:
        soc.pmod_bits_gpio.write(offset=offset, value=0x00)

In [ ]:
### Write to the user IO ports (built in port) ###

print("Reminder: port 0, 1, 2 should be set to 'write/output' (position 0)")
print("Reminder: port 3, 4, 5 should be set to 'read/input' (position 1)")

# setup read/write port map
soc.user_io_gpio.write(offset=4, value=0b111000)

print("Reminder: the output ports should be cabled to the input ports")

try:
    for value in tqdm([0x00, 0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07]): # user I/O port is 6 bits wide
        soc.user_io_gpio.write(offset=0x00, value=value)
        found_raw = soc.user_io_gpio.read(offset=0x00)
        found = (found_raw & 0b111000) >> 3
        wrote = (found & 0b000111)
        if found != value:
            raise ValueError(f"0x00 read value {hex(found)} rather than {hex(value)}")
        if wrote != value:
            raise ValueError(f"0x00 wrote value {hex(value)} but saw {hex(wrote)}")
    print("No errors found")
finally:
    soc.user_io_gpio.write(offset=0x00, value=0x00)

In [ ]:
### Print list of daughter cards and their type ###

from qick.rfboard import AdcDcCard216, AdcRfCard216, DacDcCard216, DacRfCard216

total_output_channels = list(range(16))
total_input_channels = list(range(8))

for slot, card in enumerate(soc.dac_cards):
    with soc.board_sel.enable_context(board_id=slot):
        channels = total_output_channels[slot * 4:(slot + 1) * 4]
        if card is None:
            print(f"slot {slot}: No card detected")
        elif isinstance(card, DacDcCard216):
            print(f"slot {slot}: DAC card has channels: DC {channels} (card_num:{card.card_num})")
        elif isinstance(card, DacRfCard216):
            print(f"slot {slot}: DAC card has channels: RF {channels} (card_num:{card.card_num})")
for raw_slot, card in enumerate(soc.adc_cards):
    channels = total_input_channels[raw_slot * 2:(raw_slot + 1) * 2]
    slot = raw_slot + 4
    with soc.board_sel.enable_context(board_id=slot):
        if card is None:
            print(f"slot {slot}: No card detected")
        elif isinstance(card, AdcDcCard216):
            print(f"slot {slot}: ADC card has channels: DC {channels} (card_num:{card.card_num})")
        elif isinstance(card, AdcRfCard216):
            print(f"slot {slot}: ADC card has channels: RF {channels} (card_num:{card.card_num})")

In [ ]:
### Test RF Cards ###
#
# Please set an OUTPUT_CHANNEL
# If you have no INPUT_CHANNEL please set it to None
#

RF_INPUT_CHANNEL_NUMBER = 5
RF_OUTPUT_CHANNEL_NUMBER = 12

test_rf_card(soc, RF_OUTPUT_CHANNEL_NUMBER, RF_INPUT_CHANNEL_NUMBER)

In [ ]:
### Test DC Cards ###
#
# Please set an OUTPUT_CHANNEL
# If you have no INPUT_CHANNEL please set it to None
#

DC_INPUT_CHANNEL_NUMBER = None
DC_OUTPUT_CHANNEL_NUMBER = 7

test_dc_card(soc, DC_OUTPUT_CHANNEL_NUMBER, DC_INPUT_CHANNEL_NUMBER)